This file shows how good (or bad) I marked the labels.

To check labeling I'm trying to cluster all the abilities
based on the labels they have and check results manually.

## TODO
1. (One day) Write distance function which is going to use nmf classification of abilities texts.

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer

from atod import Abilities

In [2]:
specs = Abilities.all().get_specs_list()
specs.shape

(519, 837)

In [4]:
columns_to_drop = ['lvl', 'AbilityCastRangeBuffer', 'levelkey', 
                   'HotKeyOverride', 'AbilityTextureName']
specs = specs.drop(columns_to_drop, axis=1)
specs = specs.dropna(axis=1, thresh=2)
specs = specs.fillna(value=0)
specs.shape

(519, 253)

In [5]:
# encode AbilityBehavior
# get all possible values
behavior = specs.AbilityBehavior
behavior = behavior.str.split('|').apply(lambda x: [y.strip() for y in x])

mlb = MultiLabelBinarizer().fit(behavior)
binary_behavior = pd.DataFrame(mlb.transform(behavior),
                               columns=mlb.classes_).drop([''], axis=1)

binary_behavior.head()
X = pd.concat([specs.drop(['AbilityBehavior'], axis=1), binary_behavior], axis=1)

In [6]:
str_columns = []

for column in X:
    if X[column].dtype != np.float64 and X[column].dtype != np.int64:
        str_columns.append(column)
        
print(str_columns)

['AbilityType', 'AbilityUnitDamageType', 'AbilityUnitTargetFlags', 'AbilityUnitTargetTeam', 'AbilityUnitTargetType', 'LinkedSpecialBonus', 'SpellDispellableType', 'SpellImmunityType', '_sa_instance_state', 'name', 'pk']


In [7]:
X['AbilityType'] = X['AbilityType'].apply(
                    lambda x: 1 if x == 'DOTA_ABILITY_TYPE_ULTIMATE' else 0)

X['isUltimate'] = X['AbilityType']
X = X.drop(['AbilityType'], axis=1)

In [8]:
X['AbilityUnitDamageType'] = X['AbilityUnitDamageType'].apply(
                              lambda x: 'DAMAGE_TYPE_NONE' if x is None else x)

mlb = MultiLabelBinarizer().fit(X['AbilityUnitDamageType'])
dmg_type = pd.DataFrame(mlb.transform(X['AbilityUnitDamageType']),
                        columns=mlb.classes_)

dmg_type.head()
X = pd.concat([X.drop(['AbilityUnitDamageType'], axis=1), binary_behavior], axis=1)

TypeError: 'int' object is not iterable

In [ ]:
X.head()